Learning Automaton
Forum Scrape Project
Information Processing

In [ ]:
import pymongo

from pprint import pprint
from pymongo import MongoClient
client = MongoClient('192.168.2.70', 27017)
db=client['fruitsandveggies']
collection = db['aggregate-posts-out_2']

In [ ]:
import spacy
from spacy import displacy

# TODO test lg vs sm
# TODO test spacy core in binder
nlp = spacy.load('en_core_web_lg')
#nlp = spacy.load('en_core_web_sm')


In [ ]:
posts = collection.find({})
raw = '. '.join([post['post_content'] for post in posts if post['post_content'] != "\\n"])
raw = raw.replace("\n"," ")

In [ ]:
#raw = """Apples are the best. Us enlightened who call them, 'pulls love them more than any other fruit. 'Pulls are tasty, sweet, and crisp. All other fruits and those yucky vegetables are inferior. Discuss.. Ya, 'pulls are the best. 'Pulls are ripe, juicy, and taste sooooo good. Nothing else can match! I'd eat apples every day all day if I could, doctors be damned!. Yall get real. Apples are gross. The worst of the fruits, which are all bad. Everyone knows that veggies are powerfood. Get your week apple-sauce out of here.. Everyone knows grapes are superior you filthy casual. Enjoy your cyanide balls. Battlestar Galactica.. NO ONE READ THE ABOVE. THAT IS NOT ME.  Identity theft is not a joke, Jim! Millions of families suffer every year!. MICHAEL!  *storms off*. Oh, that's funny.  MICHAEL. From my head tomatoes.. Never Leaf Me.  We make a pearfect couple.. Peaches are better than all others. Peaches are the best, cabbage is the worst, end of story.  Smelly Cabbage &gt;.&lt; yuck. Corn cob rows!  Take me home,  to the placeeee,  I belonggg!!!!  WEST INDIANAAAAA  BUTTER MAMAAAAA  TAKE ME HOMEEEEE  CORN BOB ROWWWSS. If a squash can make you smile.... Lol, tomatoes and squash are both fruits! Get this compost out of the veggie master plate forum.. Na, tomatoes and squash are fruits in name only. Acidic tomatoes and dirty squash, just like dirty tomatoes, should stay out of our sweet fruit forum. Like yucky lettuce, they belong in the vegetable forum, aka the compost.. Knock Knock. Who's There?. Banana. Banana who?. Knock Knock.... Who's There?... Banana. Banana who?. Knock Knock. Whos. ... There..... Orange!. Orange who?. ORANGE YOU GLAD I DIDN'T SAY BANANA??!?. You thought you'd pull a fast one on me, you little bitch? I'll have you know I graduated top of my class in the Tasty Peels, and I've been involved in numerous secret raids on Al-Quinoa, and I have over 300 confirmed spills. I am trained in vanilla warfare and I'm the top ripener in the entire US farmed forces. You are nothing to me but just another target grocery aisle. I will wipe you the fuck out with precision the likes of which has never been seen before on this Earth, mark my fucking words. You think you can get away with saying that shit to me over the Internet? Think again, fucker. As we speak I am contacting my secret network of pies across the USA and your IP is being traced right now so you better prepare for the storm, maggot. The storm that wipes out the pathetic little thing you call your life. You're fucking dead, kid. I can be anywhere, anytime, and I can eat you in over seven hundred ways, and that's just with my bare hands. Not only am I extensively trained in organic combat, but I have access to the entire pesticide arsenal of the United States Soybean Crops and I will use it to its full extent to wipe your miserable ass off the face of the continent, you little shit. If only you could have known what appleholey retribution your little "clever" comment was about to bring down upon you, maybe you would have held your fucking tongue. But you couldn't, you didn't, and now you're paying the price, you goddamn idiot. I will shit fury all over you and you will drown in it. Your lettuce head is fucking dead, kiddo.. Get boiled, damn."""

In [ ]:
doc = nlp(raw)


In [ ]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

noun_is_adj = [{"POS": "NOUN"}, {"LEMMA": "be"}, {"POS": "ADV", "OP": "*"}, {"POS": "DET", "OP": "*"},
           {"POS": "ADJ", "OP": "+"}]

adj_noun = [ {"POS": "ADJ"},{"POS": "NOUN"}]

matcher.add("NounIsAdj", None, noun_is_adj)  # add pattern
matcher.add("AdjNoun", None, adj_noun)

matches = matcher(doc)

for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

In [ ]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

In [ ]:

noun_adj_pairs = []
for i,token in enumerate(doc):
    if token.pos_ not in ('NOUN','PROPN'):
        continue
    for j in range(i+1,len(doc)):
        if doc[j].pos_ == 'ADJ':
            noun_adj_pairs.append((token,doc[j]))
            break
noun_adj_pairs

In [ ]:
from spacy.lang.en import English
from spacy.pipeline import EntityRuler

ruler = EntityRuler(nlp)
patterns = [{"label": "FRUIT", "pattern": [{"lower":"apple"}]},
            {"label": "FRUIT", "pattern": [{"lower":"grape"}]},
           ]
ruler.add_patterns(patterns)
nlp.add_pipe(ruler)

threads = collection.find({"title":"Apples ('pulls) are the best fruit"})
threads = [thread['posts'] for thread in threads]
posts = [post for thread in threads for post in thread]
contents = [post['content'] for post in posts]
#pprint(contents)

